In [ ]:
import cv2
import os
import requests
from bs4 import BeautifulSoup

def get_umc_profiles(url):
    """Scrapes UMC profiles from the given URL."""
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.content, 'html.parser')

        profiles = []
        for item in soup.find_all('div', class_='umc-item'):
            name = item.find('h3').text.strip()
            title = item.find('p', class_='umc-title').text.strip()
            image_url = item.find('img')['src']

            # Extract first and last names
            names = name.split()
            first_name = names[0]
            last_name = " ".join(names[1:]) #handles multiple last names

            profiles.append({
                'name': name,
                'title': title,
                'image_url': image_url,
                'first_name': first_name,
                'last_name': last_name
            })
        return profiles

    except requests.exceptions.RequestException as e:
        print(f"Error fetching UMC profiles: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None



def download_image(url, filename):
    """Downloads an image from the given URL."""
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        return True
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")
        return False

def calculate_password(last_name):
    """Calculates the password based on the length of the last name."""
    return len(last_name.replace(" ", "")) #removes spaces for multi last names

def face_recognition_login(profiles):
    """Facial recognition login function."""

    username = input("Enter your username (first name): ").strip()
    password = input("Enter your password: ").strip()

    user_profile = None
    for profile in profiles:
        if profile['first_name'].lower() == username.lower():
            user_profile = profile
            break

    if not user_profile:
        print("User not found.")
        return

    expected_password = calculate_password(user_profile['last_name'])
    if str(expected_password) != password: #convert to string for comparison
        print("Incorrect password.")
        return

    image_filename = f"{user_profile['first_name']}_{user_profile['last_name'].replace(' ', '_')}.jpg" #handles multiple last names in filename

    if not os.path.exists(image_filename):
        if not download_image(user_profile['image_url'], image_filename):
            return

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    img = cv2.imread(image_filename)

    if img is None:
        print("Error loading image for facial recognition.")
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) > 0:
        print(f"Login successful! Welcome, {user_profile['name']}.")
        print(f"Title: {user_profile['title']}")
        cv2.imshow('Profile Image', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("No face detected in the image. Login failed.")

# Main execution
umc_url = "https://pau.edu.ng/about-pau/governance/umc/"
umc_profiles = get_umc_profiles(umc_url)

if umc_profiles:
    face_recognition_login(umc_profiles)